In [1]:
print("Importing libraries...")
import csv
import os
import pandas as pd
import numpy as np
import random

import re
from urllib.parse import urlparse

import logging

random.seed(42)

print("Libraries imported!")


pd.options.display.float_format = "{:.2f}".format

# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

Importing libraries...
Libraries imported!


In [2]:
url = r"C:\Users\owner\Desktop\Files_Deep_Learning\NeuraGuide\AI_Tools.csv"

def load_data(path):
    data = pd.read_csv(path)
    return data


df = load_data(url)
df = df.copy()

In [3]:
df.columns

Index(['Tool Name', 'Category', 'Primary Function', 'Description', 'Website',
       'Pricing Model', 'Key Features', 'Target Users', 'Launch Year',
       'Company', 'category_rank', 'ID', 'Category_code', 'average_rating',
       'review_count'],
      dtype='object')

In [4]:
df.head()

,Tool Name,Category,Primary Function,Description,Website,Pricing Model,Key Features,Target Users,Launch Year,Company,category_rank,ID,Category_code,average_rating,review_count
0,Kaedim,3D,3D,Transform 2D images into high-quality 3D model...,https://www.kaedim3d.com,Paid,See website,General,Unknown,Unknown,1,1,0,0.00,0
1,Kinetix,3D,3D,"Revolutionize 3D animation with AI, democratiz...",https://www.kinetix.tech,Paid,See website,General,Unknown,Unknown,2,2,0,0.00,0
2,GET3D by NVIDIA,3D,3D,"Revolutionize 3D modeling with AI-powered, tex...",https://nv-tlabs.github.io,Free,See website,General,Unknown,Unknown,3,3,0,0.00,0
3,DeepMotion,3D,3D,Revolutionize animation with AI-driven motion ...,https://www.deepmotion.com,Freemium,See website,General,Unknown,Unknown,4,4,0,0.00,0
4,Wonder Studio,3D,3D,"Revolutionize VFX: automate animation, lightin...",https://wonderdynamics.com,Freemium,See website,General,Unknown,Unknown,5,5,0,0.00,0


In [5]:
df.shape

(54910, 15)

In [6]:
def check_missing_values(df):
    missing_values = df.isnull().sum()
    if missing_values.sum()>0:
        print(missing_values[missing_values>0])
    else:
        print("No missing values")

check_missing_values(df)

Launch Year    1420
dtype: int64


In [7]:

df["Launch Year"] = np.where(df["Launch Year"] == "Unknown", 
                             np.random.randint(2020, 2025, size=len(df)), 
                             df["Launch Year"]) 

# df['Launch Year'] = df['Launch Year'].astype('Int64')

df["average_rating"] = np.where(df["average_rating"] == 0.0, 
                             np.round(1 + 4 * np.random.beta(a=5, b=1.5, size=len(df)), 2), 
                             df["average_rating"]) 

mask = df["Company"] == "Unknown"
df.loc[mask, "Company"] = df.loc[mask, "Tool Name"]

In [8]:
df.head(1)

,Tool Name,Category,Primary Function,Description,Website,Pricing Model,Key Features,Target Users,Launch Year,Company,category_rank,ID,Category_code,average_rating,review_count
0,Kaedim,3D,3D,Transform 2D images into high-quality 3D model...,https://www.kaedim3d.com,Paid,See website,General,2022,Kaedim,1,1,0,4.26,0


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54910 entries, 0 to 54909
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Tool Name         54910 non-null  object 
 1   Category          54910 non-null  object 
 2   Primary Function  54910 non-null  object 
 3   Description       54910 non-null  object 
 4   Website           54910 non-null  object 
 5   Pricing Model     54910 non-null  object 
 6   Key Features      54910 non-null  object 
 7   Target Users      54910 non-null  object 
 8   Launch Year       53490 non-null  object 
 9   Company           54910 non-null  object 
 10  category_rank     54910 non-null  int64  
 11  ID                54910 non-null  int64  
 12  Category_code     54910 non-null  int64  
 13  average_rating    54910 non-null  float64
 14  review_count      54910 non-null  int64  
dtypes: float64(1), int64(4), object(10)
memory usage: 6.3+ MB


In [10]:
def check_missing_values(df):
    missing_values = df.isnull().sum()
    if missing_values.sum()>0:
        print(missing_values[missing_values>0])
    else:
        print("No missing values")

check_missing_values(df)

Launch Year    1420
dtype: int64


## Identify and Remove Duplicate Tools

In [11]:
def check_duplicates(df):
    duplicates = df.duplicated().sum()
    if duplicates > 0:
        print(f"Duplicated rows: {duplicates}")
        # df = df.drop_duplicates(inplace=True)
        # dropped = df.duplicated().sum()
        # print(f"\nDuplicates: {dropped}")
    else:
        print("There are no duplicates")

check_duplicates(df)

There are no duplicates


In [12]:
# Detect duplicate entries using Tool Name, Company, and Website. Decide which record to keep and remove the rest using deterministic logic.

class DuplicateHandler:
    def __init__(self, df):
        self.df = df
        self.duplicate_keys = ['Tool Name', 'Company', 'Website']
        logger.info(f"DuplicateHandler initialized with {len(self.df)} records")
        logger.info(f"Duplicate detection keys: {', '.join(self.duplicate_keys)}")
        
    def find_duplicates(self):
        """Identify duplicate groups"""
        duplicates = self.df[self.df.duplicated(subset=self.duplicate_keys, keep=False)]
        logger.info(f"Found {len(duplicates)} duplicate records in {duplicates.groupby(self.duplicate_keys, dropna=False).ngroups} groups")
        return duplicates
    
    def rank_records(self, group):
        """Score records: higher is better"""
        scores = pd.DataFrame(index=group.index)
        scores['completeness'] = group.notna().sum(axis=1)
        scores['reviews'] = group['review_count'].fillna(0)
        scores['rating'] = group['average_rating'].fillna(0)
        scores['recency'] = group['Launch Year'].fillna(0)
        scores['position'] = -np.arange(len(group))  # negative for ascending
        return scores.sum(axis=1).idxmax()
    
    def remove_duplicates(self):
        """Keep best record per duplicate group"""
        dupes = self.find_duplicates()
        if dupes.empty:
            logger.info("No duplicates found")
            return self.df, pd.DataFrame()
        
        else:
            keep_indices = dupes.groupby(self.duplicate_keys, dropna=False).apply(self.rank_records)
            removed = self.df[self.df.index.isin(dupes.index) & ~self.df.index.isin(keep_indices)]
            cleaned = self.df[~self.df.index.isin(dupes.index) | self.df.index.isin(keep_indices)]
            
            logger.info(f"Removed {len(removed)} duplicate records")
            logger.info(f"Retained {len(cleaned)} unique records")
            return cleaned.reset_index(drop=True), removed

# Usage:
handler = DuplicateHandler(df)
cleaned_df, removed_records = handler.remove_duplicates()

2026-02-12 23:33:12,369 - INFO - DuplicateHandler initialized with 54910 records
2026-02-12 23:33:12,371 - INFO - Duplicate detection keys: Tool Name, Company, Website
2026-02-12 23:33:12,629 - INFO - Found 0 duplicate records in 0 groups
2026-02-12 23:33:12,635 - INFO - No duplicates found


In [13]:
cleaned_df.shape

(54910, 15)

In [14]:


# cleaned_df["Launch Year"] = np.where(cleaned_df["Launch Year"] == "Unknown", 
#                              np.random.randint(2020, 2025, size=len(cleaned_df)), 
#                              cleaned_df["Launch Year"]) 

# cleaned_df["average_rating"] = np.where(cleaned_df["average_rating"] == 0.0, 
#                              np.round(1 + 4 * np.random.beta(a=5, b=1.5, size=len(cleaned_df)), 2), 
#                              cleaned_df["average_rating"]) 

# mask = cleaned_df["Company"] == "Unknown"
# cleaned_df.loc[mask, "Company"] = cleaned_df.loc[mask, "Tool Name"]


In [15]:
def check_missing_values(df):
    missing_values = df.isnull().sum()
    if missing_values.sum()>0:
        print(missing_values[missing_values>0])
    else:
        print("No missing values")

check_missing_values(df)

Launch Year    1420
dtype: int64


In [16]:
cleaned_df.head()

,Tool Name,Category,Primary Function,Description,Website,Pricing Model,Key Features,Target Users,Launch Year,Company,category_rank,ID,Category_code,average_rating,review_count
0,Kaedim,3D,3D,Transform 2D images into high-quality 3D model...,https://www.kaedim3d.com,Paid,See website,General,2022,Kaedim,1,1,0,4.26,0
1,Kinetix,3D,3D,"Revolutionize 3D animation with AI, democratiz...",https://www.kinetix.tech,Paid,See website,General,2024,Kinetix,2,2,0,3.71,0
2,GET3D by NVIDIA,3D,3D,"Revolutionize 3D modeling with AI-powered, tex...",https://nv-tlabs.github.io,Free,See website,General,2024,GET3D by NVIDIA,3,3,0,3.63,0
3,DeepMotion,3D,3D,Revolutionize animation with AI-driven motion ...,https://www.deepmotion.com,Freemium,See website,General,2024,DeepMotion,4,4,0,3.60,0
4,Wonder Studio,3D,3D,"Revolutionize VFX: automate animation, lightin...",https://wonderdynamics.com,Freemium,See website,General,2024,Wonder Studio,5,5,0,4.26,0


## Identify rows missing essential information such as Tool Name, Category, or Website. Flag or remove records based on defined rules.

In [17]:


class MissingDataHandler:
    def __init__(self, df):
        self.df = df.copy()
        # self.required_fields = required_fields or ['Tool Name', 'Category', 'Website']
        
    def find_missing(self):
        """Find rows with missing required fields"""
        mask = self.df.isna().any(axis=1)
        return self.df[mask]
    
    def get_summary(self):
        """Get missing data summary"""
        summary = pd.DataFrame({
            'missing_count': self.df.isna().sum(),
            'missing_pct': (self.df.isna().sum() / len(self.df) * 100).round(2)
        })
        return summary[summary['missing_count'] > 0].sort_values('missing_count', ascending=False)
    
    def flag_records(self):
        """Add flag column for rows with missing data"""
        flagged = self.df.copy()
        flagged['has_missing'] = self.df.isna().any(axis=1)
        logger.info(f"Flagged {flagged['has_missing'].sum()} records with missing values")
        return flagged
    
    def remove_records(self):
        """Remove rows with missing required fields"""
        missing = self.find_missing()
        cleaned = self.df[~self.df.index.isin(missing.index)]
        logger.info(f"Removed {len(missing)} rows with missing values")
        return cleaned.reset_index(drop=True), missing

# Usage:
handler = MissingDataHandler(cleaned_df)
print(handler.get_summary())
flagged_df = handler.flag_records()
# # or
cleaned_df, removed = handler.remove_records()

             missing_count  missing_pct
Launch Year           1420         2.59


2026-02-12 23:33:13,609 - INFO - Flagged 1420 records with missing values
2026-02-12 23:33:13,770 - INFO - Removed 1420 rows with missing values


In [18]:
def check_missing_values(df):
    missing_values = df.isnull().sum()
    if missing_values.sum()>0:
        print(missing_values[missing_values>0])
    else:
        print("No missing values")

check_missing_values(cleaned_df)

No missing values


In [ ]:
cleaned_df['Launch Year'] = pd.to_numeric(cleaned_df['Launch Year'], errors='coerce').astype('int64')

In [30]:
def check_missing_values(df):
    missing_values = df.isnull().sum()
    if missing_values.sum()>0:
        print(missing_values[missing_values>0])
    else:
        print("No missing values")

check_missing_values(cleaned_df)

No missing values


## 

## Validate Website: URL format Dead / malformed links

In [20]:
class URLValidator:
    def __init__(self, df, url_column='Website'):
        self.df = df.copy()
        self.url_column = url_column
        logger.info(f"URLValidator initialized for column: {url_column}")
        logger.info(f"Total records to validate: {len(self.df)}")
        
    def is_valid_url(self, url):
        """Check if URL is properly formatted"""
        if pd.isna(url) or not isinstance(url, str):
            return False
        
        # Basic pattern check
        url = url.strip()
        if not re.match(r'^https?://', url, re.IGNORECASE):
            url = 'http://' + url
        
        try:
            import validators
            return validators.url(url) is True
        except ImportError:
            # Fallback to regex if validators not installed
            pattern = re.compile(
                r'^https?://'
                r'(?:(?:[A-Z0-9](?:[A-Z0-9-]{0,61}[A-Z0-9])?\.)+[A-Z]{2,6}\.?|'
                r'localhost|'
                r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})'
                r'(?::\d+)?'
                r'(?:/?|[/?]\S+)$', re.IGNORECASE)
            return bool(pattern.match(url))
    
    def validate_urls(self):
        """Validate all URLs and return results"""
        results = self.df.copy()
        results['url_valid'] = results[self.url_column].apply(self.is_valid_url)
        results['url_missing'] = results[self.url_column].isna()

        valid_count = results['url_valid'].sum()
        missing_count = results['url_missing'].sum()
        invalid_count = (~results['url_valid'] & ~results['url_missing']).sum()

        logger.info(f"URL validation complete:")
        logger.info(f"  - Valid URLs: {valid_count}")
        logger.info(f"  - Missing URLs: {missing_count}")
        logger.info(f"  - Invalid URLs: {invalid_count}")
        return results
    
    def get_invalid_urls(self):
        """Get records with invalid URLs"""
        validated = self.validate_urls()
        return validated[~validated['url_valid'] & ~validated['url_missing']]
    
    def check_reachability(self, timeout=5, max_workers=10):
        """Check if URLs are reachable (optional)"""
        import requests
        from concurrent.futures import ThreadPoolExecutor, as_completed
        
        def check_url(url):
            if pd.isna(url):
                return None
            try:
                url = url.strip()
                if not re.match(r'^https?://', url, re.IGNORECASE):
                    url = 'http://' + url
                response = requests.head(url, timeout=timeout, allow_redirects=True)
                return response.status_code < 400
            except: 
                return False
        
        urls = self.df[self.url_column].dropna().unique()
        reachability = {}
        
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            futures = {executor.submit(check_url, url): url for url in urls}
            for future in as_completed(futures):
                url = futures[future]
                reachability[url] = future.result()
        
        results = self.df.copy()
        results['url_reachable'] = results[self.url_column].map(reachability)
        return results
    
    def clean_urls(self):
        """Remove records with invalid URLs"""
        validated = self.validate_urls()
        cleaned = validated[validated['url_valid'] | validated['url_missing']]
        invalid = validated[~validated['url_valid'] & ~validated['url_missing']]

        logger.info(f"Removed {len(invalid)} records with invalid URLs")
        logger.info(f"Retained {len(cleaned)} records with valid or missing URLs")
        
        return cleaned.drop(columns=['url_valid', 'url_missing']).reset_index(drop=True), invalid
    
    
# Usage:
validator = URLValidator(cleaned_df)
print(f"Invalid URLs: {len(validator.get_invalid_urls())}")
cleaned_df, invalid_urls = validator.clean_urls()
# 
# # Optional: Check reachability (slower), had to stop it because it took over 11mins and it wasn't done
# reachable_df = validator.check_reachability()
# print(reachable_df)

2026-02-12 23:33:14,194 - INFO - URLValidator initialized for column: Website
2026-02-12 23:33:14,200 - INFO - Total records to validate: 53490
2026-02-12 23:33:20,056 - INFO - URL validation complete:
2026-02-12 23:33:20,058 - INFO -   - Valid URLs: 53489
2026-02-12 23:33:20,063 - INFO -   - Missing URLs: 0
2026-02-12 23:33:20,065 - INFO -   - Invalid URLs: 1


Invalid URLs: 1


2026-02-12 23:33:23,547 - INFO - URL validation complete:
2026-02-12 23:33:23,548 - INFO -   - Valid URLs: 53489
2026-02-12 23:33:23,549 - INFO -   - Missing URLs: 0
2026-02-12 23:33:23,551 - INFO -   - Invalid URLs: 1
2026-02-12 23:33:23,570 - INFO - Removed 1 records with invalid URLs
2026-02-12 23:33:23,571 - INFO - Retained 53489 records with valid or missing URLs


In [21]:
cleaned_df.head()

,Tool Name,Category,Primary Function,Description,Website,Pricing Model,Key Features,Target Users,Launch Year,Company,category_rank,ID,Category_code,average_rating,review_count
0,Kaedim,3D,3D,Transform 2D images into high-quality 3D model...,https://www.kaedim3d.com,Paid,See website,General,2022,Kaedim,1,1,0,4.26,0
1,Kinetix,3D,3D,"Revolutionize 3D animation with AI, democratiz...",https://www.kinetix.tech,Paid,See website,General,2024,Kinetix,2,2,0,3.71,0
2,GET3D by NVIDIA,3D,3D,"Revolutionize 3D modeling with AI-powered, tex...",https://nv-tlabs.github.io,Free,See website,General,2024,GET3D by NVIDIA,3,3,0,3.63,0
3,DeepMotion,3D,3D,Revolutionize animation with AI-driven motion ...,https://www.deepmotion.com,Freemium,See website,General,2024,DeepMotion,4,4,0,3.60,0
4,Wonder Studio,3D,3D,"Revolutionize VFX: automate animation, lightin...",https://wonderdynamics.com,Freemium,See website,General,2024,Wonder Studio,5,5,0,4.26,0


##

In [22]:
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53489 entries, 0 to 53488
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Tool Name         53489 non-null  object 
 1   Category          53489 non-null  object 
 2   Primary Function  53489 non-null  object 
 3   Description       53489 non-null  object 
 4   Website           53489 non-null  object 
 5   Pricing Model     53489 non-null  object 
 6   Key Features      53489 non-null  object 
 7   Target Users      53489 non-null  object 
 8   Launch Year       53489 non-null  object 
 9   Company           53489 non-null  object 
 10  category_rank     53489 non-null  int64  
 11  ID                53489 non-null  int64  
 12  Category_code     53489 non-null  int64  
 13  average_rating    53489 non-null  float64
 14  review_count      53489 non-null  int64  
dtypes: float64(1), int64(4), object(10)
memory usage: 6.1+ MB


In [24]:
# condition = cleaned_df["Launch Year"]<2019
# filtered = cleaned_df[condition]
# print(filtered)

# # print(cleaned_df[cleaned_df["Launch Year"]<2019])

In [29]:
# df['Launch Year'] = pd.to_numeric(df['Launch Year'], errors='coerce').astype('int64')


In [26]:
def check_missing_values(df):
    missing_values = df.isnull().sum()
    if missing_values.sum()>0:
        print(missing_values[missing_values>0])
    else:
        print("No missing values")

check_missing_values(cleaned_df)

No missing values


## Validate Launch Year: Range checks  Missing or future values

In [ ]:
class YearValidator:
    def __init__(self, df, year_column='Launch Year', min_year=2019, max_year=2025):
        self.df = df.copy()
        self.year_column = year_column
        self.min_year = min_year
        self.max_year = max_year or pd.Timestamp.now().year
        logger.info(f"YearValidator initialized")
        
    def is_valid_year(self, year):
        """Check if year is numeric and within range"""

        if pd.isna(year):
            return None  # Missing, not invalid
        try:
            year_int = int(float(year))
            return self.min_year <= year_int <= self.max_year
        except (ValueError, TypeError):
            return False
    
    def validate_years(self):
        """Validate all years and categorize issues"""
        results = self.df.copy()
        results['year_missing'] = results[self.year_column].isna()
        results['year_valid'] = results[self.year_column].apply(self.is_valid_year)
        
        # Categorize invalid reasons
        def categorize_invalid(row):
            if row['year_missing']:
                return 'missing'
            if row['year_valid']:
                return 'valid'
            try:
                year_int = int(float(row[self.year_column]))
                if year_int > self.max_year:
                    return 'future'
                if year_int < self.min_year:
                    return 'too_old'
            except:
                return 'non_numeric'
            return 'invalid'
        
        results['year_issue'] = results.apply(categorize_invalid, axis=1)
        logger.info(f"Year validation complete: {len(results)} records processed")
        return results
    
    def get_invalid_years(self):
        """Get records with invalid years"""
        validated = self.validate_years()
        return validated[validated['year_issue'].isin(['future', 'too_old', 'non_numeric'])]
    
    def get_summary(self):
        """Summary of year validation issues"""
        validated = self.validate_years()
        summary = validated['year_issue'].value_counts()
        logger.info(f"Year validation summary:\n{summary}")
        return summary
    
    def clean_years(self, strategy='remove'):
        """
        Clean invalid years
        strategy: 'remove' (delete rows) or 'nullify' (set to NaN)
        """
        validated = self.validate_years()
        
        if strategy == 'remove':
            cleaned = validated[validated['year_issue'].isin(['valid', 'missing'])]
            invalid = validated[~validated['year_issue'].isin(['valid', 'missing'])]
        elif strategy == 'nullify':
            cleaned = validated.copy()
            mask = ~cleaned['year_issue'].isin(['valid', 'missing'])
            cleaned.loc[mask, self.year_column] = np.nan
            invalid = validated[mask]
        else:
            raise ValueError("strategy must be 'remove' or 'nullify'")
        
        # Drop helper columns
        cols_to_drop = ['year_missing', 'year_valid', 'year_issue']
        cleaned = cleaned.drop(columns=cols_to_drop)
        logger.info(f"Cleaning complete: {len(cleaned)} records remaining")
        return cleaned.reset_index(drop=True), invalid
    
    def correct_years(self):
        """Attempt to correct obvious errors"""
        corrected = self.df.copy()
        corrections = []
        
        for idx, row in corrected.iterrows():
            year = row[self.year_column]
            if pd.isna(year):
                continue
                
            try:
                year_int = int(float(year))
                original = year_int
                
                # Common errors: 2-digit years
                if 0 <= year_int <= 99:
                    year_int = 2000 + year_int if year_int <= self.max_year % 100 else 1900 + year_int
                
                # Future years: might be typo (e.g., 2025 instead of 2015)
                if year_int > self.max_year:
                    continue  # Can't reliably correct
                
                if year_int != original and self.min_year <= year_int <= self.max_year:
                    corrected.at[idx, self.year_column] = year_int
                    corrections.append({
                        'index': idx,
                        'original': original,
                        'corrected': year_int,
                        'tool': row.get('Tool Name', 'Unknown')
                    })
            except:
                continue
        logger.info(f"Corrected {len(corrections)} year values")
        return corrected, pd.DataFrame(corrections)


validator = YearValidator(cleaned_df)
validator.get_summary()
cleaned_df, invalid_records = validator.clean_years(strategy='nullify')
# 
# # Or attempt corrections first
# corrected_df, corrections_log = validator.correct_years()

2026-02-12 23:34:24,953 - INFO - YearValidator initialized


IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

In [ ]:
cleaned_df.columns

Index(['Tool Name', 'Category', 'Primary Function', 'Description', 'Website',
       'Pricing Model', 'Key Features', 'Target Users', 'Launch Year',
       'Company', 'category_rank', 'ID', 'Category_code', 'average_rating',
       'review_count'],
      dtype='object')

## Validate numeric columns: average_rating bounds  review_count non-negativity

In [ ]:
class NumericValidator:
    def __init__(self, df, rating_col='average_rating', review_col='review_count', 
                 rating_min=0, rating_max=5):
        self.df = df.copy()
        self.rating_col = rating_col
        self.review_col = review_col
        self.rating_min = rating_min
        self.rating_max = rating_max
        
    def validate_ratings(self):
        """Check if ratings are within valid bounds"""
        ratings = self.df[self.rating_col].copy()
        mask = (ratings < self.rating_min) | (ratings > self.rating_max)
        return self.df[mask & ratings.notna()]
    
    def validate_reviews(self):
        """Check if review counts are non-negative integers"""
        reviews = self.df[self.review_col].copy()
        
        # Check for negative or non-integer values
        mask_negative = reviews < 0
        mask_non_integer = reviews != reviews.astype(int)
        mask = (mask_negative | mask_non_integer) & reviews.notna()
        
        return self.df[mask]
    
    def get_summary(self):
        """Get validation summary"""
        invalid_ratings = len(self.validate_ratings())
        invalid_reviews = len(self.validate_reviews())
        
        summary = {
            'invalid_ratings': invalid_ratings,
            'invalid_reviews': invalid_reviews,
            'total_rows': len(self.df)
        }
        
        logger.info(f"Invalid ratings: {invalid_ratings}")
        logger.info(f"Invalid review counts: {invalid_reviews}")
        
        return summary
    
    def flag_records(self):
        """Add flag columns for invalid values"""
        flagged = self.df.copy()
        
        ratings = flagged[self.rating_col]
        reviews = flagged[self.review_col]
        
        flagged['invalid_rating'] = ((ratings < self.rating_min) | (ratings > self.rating_max)) & ratings.notna()
        flagged['invalid_review'] = ((reviews < 0) | (reviews != reviews.astype(int))) & reviews.notna()
        
        return flagged
    
    def clean_records(self, strategy='remove'):
        """
        Clean invalid records
        strategy: 'remove' (delete rows) or 'nullify' (set to NaN)
        """
        invalid_ratings = self.validate_ratings()
        invalid_reviews = self.validate_reviews()
        invalid_indices = invalid_ratings.index.union(invalid_reviews.index)
        
        if strategy == 'remove':
            cleaned = self.df[~self.df.index.isin(invalid_indices)]
            invalid = self.df[self.df.index.isin(invalid_indices)]
        elif strategy == 'nullify':
            cleaned = self.df.copy()
            
            ratings = cleaned[self.rating_col]
            mask_rating = ((ratings < self.rating_min) | (ratings > self.rating_max)) & ratings.notna()
            cleaned.loc[mask_rating, self.rating_col] = np.nan
            
            reviews = cleaned[self.review_col]
            mask_review = ((reviews < 0) | (reviews != reviews.astype(int))) & reviews.notna()
            cleaned.loc[mask_review, self.review_col] = np.nan
            
            invalid = self.df[self.df.index.isin(invalid_indices)]
        else:
            raise ValueError("strategy must be 'remove' or 'nullify'")
        
        logger.info(f"Cleaned {len(invalid)} records with invalid numeric values")
        return cleaned.reset_index(drop=True), invalid


validator = NumericValidator(cleaned_df, rating_min=0, rating_max=5)
print(validator.get_summary())
flagged_df = validator.flag_records()
# # or
cleaned_df, invalid = validator.clean_records(strategy='nullify')

2026-02-12 23:15:13,325 - INFO - Invalid ratings: 0


2026-02-12 23:15:13,327 - INFO - Invalid review counts: 0
2026-02-12 23:15:13,368 - INFO - Cleaned 0 records with invalid numeric values


{'invalid_ratings': 0, 'invalid_reviews': 0, 'total_rows': 53489}


## Normalize text fields: Casing  Whitespace  Encoding issues

In [ ]:
class TextStandardizer:
    def __init__(self, df, text_columns=None):
        self.df = df.copy()
        self.text_columns = text_columns or self.df.select_dtypes(include=['object']).columns.tolist()
        
    def clean_text(self, text):
        """Standardize single text value"""
        if pd.isna(text) or not isinstance(text, str):
            return text
        
        # Remove extra whitespace
        text = ' '.join(text.split())
        
        # Strip leading/trailing whitespace
        text = text.strip()
        
        # Normalize unicode characters
        text = text.encode('utf-8', errors='ignore').decode('utf-8')
        
        return text
    
    
    def standardize_all(self, case_rules=None):
        """
        Standardize all text columns
        case_rules: dict mapping column names to case types
        """
        standardized = self.df.copy()
        
        # Default case rules
        if case_rules is None:
            case_rules = {
                'Tool Name': 'title',
                'Category': 'title',
                'Company': 'title',
                'Primary Function': 'sentence',
                'Description': 'sentence',
                'Pricing Model': 'title',
                'Target Users': 'title'
            }
        
        # Clean whitespace and encoding for all text columns
        for col in self.text_columns:
            if col in standardized.columns:
                standardized[col] = standardized[col].apply(self.clean_text)
        
        # Apply case rules
        for col, case_type in case_rules.items():
            if col in standardized.columns:
                if case_type == 'title':
                    standardized[col] = standardized[col].str.title()
                elif case_type == 'lower':
                    standardized[col] = standardized[col].str.lower()
                elif case_type == 'upper':
                    standardized[col] = standardized[col].str.upper()
                elif case_type == 'sentence':
                    standardized[col] = standardized[col].apply(
                        lambda x: x.capitalize() if isinstance(x, str) else x
                    )
        
        logger.info(f"Standardized {len(self.text_columns)} text columns")
        return standardized
    
    def get_changes_summary(self, standardized_df):
        """Compare original vs standardized data"""
        changes = []
        
        for col in self.text_columns:
            if col in self.df.columns:
                mask = self.df[col] != standardized_df[col]
                changed_count = mask.sum()
                if changed_count > 0:
                    changes.append({
                        'column': col,
                        'changes': changed_count,
                    })
        
        return pd.DataFrame(changes)

# Usage:
standardizer = TextStandardizer(cleaned_df)
cleaned_df = standardizer.standardize_all()
# 
# 
# # See what changed
changes = standardizer.get_changes_summary(cleaned_df)

2026-02-12 23:15:22,387 - INFO - Standardized 10 text columns


## Flag descriptions that are too short or non-informative

In [ ]:
class DescriptionValidator:
    def __init__(self, df, description_col='Description', min_length=10, min_words=3):
        self.df = df.copy()
        self.description_col = description_col
        self.min_length = min_length
        self.min_words = min_words
        self.meaningless = ['n/a', 'na', 'none', 'null', 'tbd', 'tba', 'coming soon',
                           'no description', 'not available', 'see website', 'lorem ipsum']
        
    def get_flag_reason(self, text):
        """Check description and return reason if invalid"""
        if pd.isna(text):
            return 'missing'
        
        text_str = str(text).strip()
        text_lower = text_str.lower()
        reasons = []
        
        if len(text_str) < self.min_length:
            reasons.append('too_short')
        if len(text_str.split()) < self.min_words:
            reasons.append('few_words')
        if any(pattern in text_lower for pattern in self.meaningless):
            reasons.append('meaningless')
        
        return ', '.join(reasons) if reasons else None
    
    def flag_descriptions(self):
        """Add flag column for invalid descriptions"""
        flagged = self.df.copy()
        flagged['desc_flag'] = flagged[self.description_col].apply(self.get_flag_reason)
        
        invalid_count = flagged['desc_flag'].notna().sum()
        logger.info(f"Flagged {invalid_count} invalid descriptions")
        
        return flagged
    
    def get_summary(self):
        """Get summary of flagged descriptions"""
        flagged = self.flag_descriptions()
        total_flagged = flagged['desc_flag'].notna().sum()
        
        reasons = flagged['desc_flag'].dropna().str.split(', ').explode().value_counts()
        
        logger.info(f"Total flagged: {total_flagged}")
        logger.info(f"\n{reasons}")
        
        return reasons

# Usage:
validator = DescriptionValidator(cleaned_df, min_length=10, min_words=3)
summary = validator.get_summary()
flagged_df = validator.flag_descriptions()

2026-02-12 23:15:25,208 - INFO - Flagged 10485 invalid descriptions
2026-02-12 23:15:25,321 - INFO - Total flagged: 10485
2026-02-12 23:15:25,324 - INFO - 
desc_flag
meaningless    10484
few_words          1
Name: count, dtype: int64
2026-02-12 23:15:25,841 - INFO - Flagged 10485 invalid descriptions


In [ ]:
# def save_cleaned_data(data: pd.DataFrame, filename: str, index: bool = False):
#     try: 
#         data.to_csv(filename, index = False)
#         logger.info(
#             "Cleaned data saved successfully")
#     except IOError as e:
#         logger.error(
#             f"Failed to save cleaned data to {filename}",
#             exc_info=True
#         )

# save_cleaned_data(cleaned_df, filename="Cleaned_AI_Tools_final.csv")

2026-02-12 23:15:39,938 - INFO - Cleaned data saved successfully


In [ ]:
# paths = r"C:\Users\owner\Desktop\Files_Deep_Learning\NeuraGuide\Cleaned_AI_Tools.csv"
# dff = load_data(path=paths)

# dff.shape

(54909, 15)